In [1]:


import torch
import time

dist_matrix = torch.rand(1000, 120)
print(dist_matrix.shape)
num_nearest_neighbors = 5

start = time.time() 
ind = torch.topk(dist_matrix, num_nearest_neighbors, largest=False).indices
end = time.time() 

print(f"Time taken to run the function: {end - start} seconds")
print(ind.shape)


torch.Size([1000, 120])
Time taken to run the function: 0.0005297660827636719 seconds
torch.Size([1000, 5])
